In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
from datetime import datetime


In [5]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .getOrCreate()

In [8]:
# Sample data as a dictionary
data = [{
    'transactionId': '8847afe5-c86d-4e5a-be53-93c78b2cca5b', 
    'productId': 'product1', 
    'productName': 'laptop', 
    'productCategory': 'grocery', 
    'productPrice': 242.46, 
    'productQuantity': 10, 
    'productBrand': 'oneplus', 
    'currency': 'GBP', 
    'customerId': 'amanda72', 
    'transactionDate': datetime.strptime('2024-06-29T08:56:08.515900', '%Y-%m-%dT%H:%M:%S.%f'), 
    'paymentMethod': 'online_transfer'
}]

# Define schema
schema = StructType([
    StructField("transactionId", StringType(), True),
    StructField("productId", StringType(), True),
    StructField("productName", StringType(), True),
    StructField("productCategory", StringType(), True),
    StructField("productPrice", DoubleType(), True),
    StructField("productQuantity", IntegerType(), True),
    StructField("productBrand", StringType(), True),
    StructField("currency", StringType(), True),
    StructField("customerId", StringType(), True),
    StructField("transactionDate", TimestampType(), True),
    StructField("paymentMethod", StringType(), True)
])

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, schema=schema)

parsed_df  = df.selectExpr("CAST(value AS STRING) as json") \
                .select(from_json(col("json"), schema).alias("data")) \
                .select("data.*")

# Add a new column 'revenue' which is productPrice * productQuantity
enriched_df = parsed_df.withColumn("revenue", col("productPrice") * col("productQuantity"))

# Show the enriched DataFrame
enriched_df.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `value` cannot be resolved. Did you mean one of the following? [`currency`, `productId`, `customerId`, `productName`, `productPrice`].; line 1 pos 5;
'Project [cast('value as string) AS json#195]
+- LogicalRDD [transactionId#173, productId#174, productName#175, productCategory#176, productPrice#177, productQuantity#178, productBrand#179, currency#180, customerId#181, transactionDate#182, paymentMethod#183], false


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
from datetime import datetime

# Initialize Spark session
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

# Sample data as a dictionary
data = [{
    'transactionId': '8847afe5-c86d-4e5a-be53-93c78b2cca5b', 
    'productId': 'product1', 
    'productName': 'laptop', 
    'productCategory': 'grocery', 
    'productPrice': 242.46, 
    'productQuantity': 10, 
    'productBrand': 'oneplus', 
    'currency': 'GBP', 
    'customerId': 'amanda72', 
    'transactionDate': datetime.strptime('2024-06-29T08:56:08.515900', '%Y-%m-%dT%H:%M:%S.%f'), 
    'paymentMethod': 'online_transfer'
}]

# Define schema
schema = StructType([
    StructField("transactionId", StringType(), True),
    StructField("productId", StringType(), True),
    StructField("productName", StringType(), True),
    StructField("productCategory", StringType(), True),
    StructField("productPrice", DoubleType(), True),
    StructField("productQuantity", IntegerType(), True),
    StructField("productBrand", StringType(), True),
    StructField("currency", StringType(), True),
    StructField("customerId", StringType(), True),
    StructField("transactionDate", TimestampType(), True),
    StructField("paymentMethod", StringType(), True)
])

# Create a DataFrame from the sample data
df = spark.createDataFrame(data, schema=schema)
df.show()



+--------------------+---------+-----------+---------------+------------+---------------+------------+--------+----------+--------------------+---------------+
|       transactionId|productId|productName|productCategory|productPrice|productQuantity|productBrand|currency|customerId|     transactionDate|  paymentMethod|
+--------------------+---------+-----------+---------------+------------+---------------+------------+--------+----------+--------------------+---------------+
|8847afe5-c86d-4e5...| product1|     laptop|        grocery|      242.46|             10|     oneplus|     GBP|  amanda72|2024-06-29 08:56:...|online_transfer|
+--------------------+---------+-----------+---------------+------------+---------------+------------+--------+----------+--------------------+---------------+

